In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import somatotopy_python as sp
import matplotlib.pyplot as plt
import scipy
import pickle as pk

## Set up model

### Load inputs

In [ ]:
# Load input set
input_responses = np.load('inputs_uniform_tutorial_2.npy')
input_responses = input_responses.T

### Input arguments

In [ ]:
seed = None
sheet_size_a = 30
sheet_size_b = sheet_size_a
input_num = 1000 #(sheet_size_a*sheet_size_b)*10

### Create random seed

In [ ]:
if seed is not None:
    np.random.seed(seed)

### Set up cortical sheet

#### Calculate size of cortical sheet

In [ ]:
cortical_sheet_size = sheet_size_a*sheet_size_b

#### Generate 2D sheets

In [ ]:
output_X, output_Y = np.meshgrid(np.linspace(1,sheet_size_a,sheet_size_a),np.linspace(1,sheet_size_b,sheet_size_b))
output_X = output_X.flatten()
output_Y = output_Y.flatten()

### Length of hand_pop location matrix

In [ ]:
# load the sample hand pop (tutorial 1)
with open('tutorial_2_hand_pop', 'rb') as f:
    hand_pop = pk.load(f)
    
hand_variable=hand_pop.location
    
hand_pop_length = len(hand_pop.location)

### Generate random weights

#### Generate random afferent weights

In [ ]:
# initial random weights
affWeights = np.random.rand(cortical_sheet_size, hand_pop_length)

# normalise weights
affWeights = np.divide(affWeights,sum(affWeights,2))

#### Generate random lateral inhibitory and excitatory weights

In [ ]:
excitWeights = np.zeros((cortical_sheet_size,cortical_sheet_size))
inhibWeights = np.zeros((cortical_sheet_size,cortical_sheet_size))

for i in range(cortical_sheet_size):
    
    excitWeights[i] = np.random.rand(cortical_sheet_size)
    excitWeights[i] = excitWeights[i]/sum(excitWeights[i])
    
    inhibWeights[i] = np.random.rand(cortical_sheet_size)
    inhibWeights[i] = inhibWeights[i]/sum(inhibWeights[i])

### Find number of input responses

In [ ]:
number_responses = np.size(input_responses,0);

### Generate vector of inputs

In [ ]:
# vector of num_patterns length random integers, selected from number of responses
random = np.random.choice(number_responses,input_num)
inputs = input_responses[random,:]

### Homeostasis parameters

In [ ]:
beta = .991 # smoothing parameter
lmbda = .005 # homeostatic learning rate
mu = .024 # target activation value
initTheta = .01 # initial threshold value

#### Initiate dictionary to store threshold and average activation values

In [ ]:
av_act = {}
theta = {}

for i in range(cortical_sheet_size):
    av_act[i] = np.zeros(input_num)
    theta[i] = np.zeros(input_num)

## Run model

In [ ]:
# present one stimulus response per iteration to train the network
for t in range(input_num):
    
    # calculate response from afferent connections
    affContribution = np.dot(affWeights,inputs[t,:].T)
        
    activation,inner,outer = sp.initial_activations(affContribution = affContribution, excitWeights = excitWeights, inhibWeights = inhibWeights,
                                cortical_sheet_size = cortical_sheet_size, output_X=output_X,output_Y=output_Y)
    
    
    activation,theta,av_act = sp.homeostasis(t=t,theta=theta,av_act=av_act,activation=activation)
    
    affWeights,excitWeights,inhibWeights = sp.update_weights(activation=activation, inner=inner, outer=outer,
                                                         affWeights=affWeights,inhibWeights=inhibWeights,
                                                         excitWeights=excitWeights,presyn_input=inputs[t,:])

## Plot map

### Set up plotting variables

In [ ]:
# set up variables to plot the map
k = sp.hand_as_generic(hand_pop=hand_pop,color_type='old')

key_list = k[0]

cmap = k[1]

afferent_colors = k[2]

grouping_variable = k[3]

### Plot standard map

In [ ]:
map_standard = sp.view_map(variable_colors=afferent_colors,w=affWeights.T,color_map=cmap,key_list=key_list,save_name='example_map.png',ss_a=sheet_size_a,ss_b=sheet_size_b)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

### Plot WTA map

In [ ]:
w_wta = sp.wta_weights(w=affWeights.T,gv_index= grouping_variable,group_name=key_list,method='wta_1')

w_wta = w_wta[0]

# In[plot map]
map_wta = sp.view_map(variable_colors=afferent_colors,w=w_wta,color_map=cmap,key_list=key_list,ss_a=sheet_size_a,ss_b=sheet_size_b)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

### Investigation of threshold and average activaiton

#### Plot of threshold over time for unit 0

##### Currently the threshold is going below zero. 
I need to think about an appropriate starting value (initTheta),
target activation (mu) and learning rate (lambda)


In [ ]:
plt.plot(theta[0])
plt.xlabel('iteration')
plt.ylabel('threshold')

#### Plot of average activation over time for unit 0

In [ ]:
plt.plot(av_act[0])
plt.xlabel('average activation')
plt.ylabel('threshold')

#### Plot of threshold against average activation for unit 0

In [ ]:
plt.plot(theta[0],av_act[0])
#plt.axhline(y = mu, color='r', label = 'mu') #target activation
plt.xlabel('threshold')
plt.ylabel('average activation')